In [ ]:
import torch
torch.cuda.get_device_properties(0) 

In [ ]:
from transformers import AutoModelForCausalLM

model_name = "Qwen/Qwen3-8B-AWQ"

model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto")

In [ ]:
model.cuda()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

prompt = "Tell me about O'Reilly online learning"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    enable_thinking=False,
    add_generation_prompt=True
)
text

In [ ]:
import time

start = time.time()
print("\n\n*** Generate:")

tokens = tokenizer(text, return_tensors='pt')
output = model.generate(inputs=tokens.input_ids.cuda(), attention_mask=tokens.attention_mask.cuda(),
                        temperature=0.7, 
                        do_sample=True, top_p=0.95, top_k=40, 
                        max_new_tokens=512)
used = time.time() - start
tps = len(output[0]) / used
print(tokenizer.decode(output[0]))
print(f"{used} seconds, {tps} tokens/s")

In [ ]:
!nvidia-smi